In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np

# เชื่อมต่อกับ MT5
if not mt5.initialize():
    print("MT5 connection failed")
    quit()

symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_H4  # ใช้กราฟ 4 ชั่วโมง
n_candles = 5000  # ดึงข้อมูล 5000 แท่ง

# ดึงข้อมูลราคาจาก MT5
rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n_candles)
mt5.shutdown()

# แปลงเป็น DataFrame
df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)

# แสดงตัวอย่างข้อมูล
print(df.head())


In [ ]:
import gymnasium as gym
from gymnasium import spaces

class ForexTradingEnv(gym.Env):
    def __init__(self, df):
        super(ForexTradingEnv, self).__init__()

        self.df = df
        self.current_step = 0
        self.balance = 10000  # เงินเริ่มต้น
        self.position = 0  # 1 = Buy, -1 = Sell, 0 = Neutral
        self.entry_price = 0

        # Action Space (0 = Hold, 1 = Buy, 2 = Sell)
        self.action_space = spaces.Discrete(3)

        # Observation Space (OHLC + Indicators)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32
        )

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.balance = 10000
        self.position = 0
        self.entry_price = 0
        return self._next_observation(), {}

    def _next_observation(self):
        obs = self.df.iloc[self.current_step][['open', 'high', 'low', 'close', 'tick_volume']].values
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        self.current_step += 1
        reward = 0
        done = False

        # ราคาปัจจุบัน
        current_price = self.df.iloc[self.current_step]['close']

        if action == 1:  # Buy
            if self.position == 0:
                self.position = 1
                self.entry_price = current_price

        elif action == 2:  # Sell
            if self.position == 1:  # ปิด Buy Position
                reward = current_price - self.entry_price
                self.balance += reward * 100
            self.position = -1
            self.entry_price = current_price

        # เช็คว่าถึงจุดสิ้นสุดของข้อมูลหรือไม่
        if self.current_step >= len(self.df) - 1:
            done = True

        return self._next_observation(), reward, done, False, {}

    def render(self):
        print(f"Step: {self.current_step}, Balance: {self.balance}, Position: {self.position}")


In [ ]:
from stable_baselines3 import PPO

# สร้าง Environment
env = ForexTradingEnv(df)

# สร้างโมเดล PPO
model = PPO("MlpPolicy", env, verbose=1)

# ฝึกโมเดล
model.learn(total_timesteps=10000)

# บันทึกโมเดล
model.save("forex_ppo_model")


In [ ]:
# โหลดโมเดลที่ฝึกไว้
model = PPO.load("forex_ppo_model")

# ทดสอบกับ Market Data จริง
obs, _ = env.reset()
done = False

action, _states = model.predict(obs)

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    env.render()


In [ ]:
import time
import MetaTrader5 as mt5

def trade_with_mt5():
    if not mt5.initialize():
        print("MT5 connection failed")
        return

    symbol = "EURUSD"
    lot = 0.01
    deviation = 20

    # รีเซ็ต Environment
    obs, _ = env.reset()
    done = False

    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, _, _ = env.step(action)

        # ดึงราคาปัจจุบัน
        price = mt5.symbol_info_tick(symbol).ask if action == 1 else mt5.symbol_info_tick(symbol).bid

        # คำสั่งซื้อขาย
        if action == 1:  # Buy
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_BUY,
                "price": price,
                "deviation": deviation,
                "magic": 123456,
                "comment": "RL Buy",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }
        elif action == 2:  # Sell
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": lot,
                "type": mt5.ORDER_TYPE_SELL,
                "price": price,
                "deviation": deviation,
                "magic": 123456,
                "comment": "RL Sell",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

        # ส่งคำสั่งไป MT5
        if action in [1, 2]:
            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ Trade {symbol} {['HOLD', 'BUY', 'SELL'][action]} at {price}")
            else:
                print(f"❌ Trade Failed: {result.comment}")

        time.sleep(5)

    mt5.shutdown()

# เรียกใช้งานฟังก์ชันเพื่อรันเทรดอัตโนมัติ
trade_with_mt5()
